# Import dependencies and set-up

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import time
from googlemaps import Client as GoogleMaps
from pprint import pprint

# Import API key
from api_key import api_key


In [2]:
gmaps = GoogleMaps(api_key)

# Read in csv file

In [3]:
addresses = pd.read_csv("Airline_1908.csv")
addresses.head(5)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


# Set-up columns for longitude and latitude

In [4]:
addresses['long'] = ""
addresses['lat'] = ""

# Loop through the 'Location' column and get long and lat for each data point

In [5]:
for x in range(len(addresses)):
#for x in range(1901, 3901):
    geocode_result = gmaps.geocode(addresses['Location'][x])
    
    try: addresses['lat'][x] = geocode_result[0]['geometry']['location'] ['lat'] 
       
   
    except:
        print('location not found')
        
    try: addresses['long'][x] = geocode_result[0]['geometry']['location']['lng']
      
    except:
        print('location not found')    
addresses.head(5)

C:\Users\sueal\Anaconda3\envs\PythonWebMongo\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\sueal\Anaconda3\envs\PythonWebMongo\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not found
location not

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,long,lat
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",-77.0811,38.8835
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,-74.4229,39.3643
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...,-123.366,48.4284
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...,3.51562,56.511
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...,13.504,52.4368


In [6]:
#pprint(addresses)

In [7]:
empty = addresses.loc[addresses["lat"] != "", :]

In [8]:
empty

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,long,lat
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",-77.0811,38.8835
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,-74.4229,39.3643
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...,-123.366,48.4284
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...,3.51562,56.511
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...,13.504,52.4368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,05/20/2009,06:30,"Near Madiun, Indonesia",Military - Indonesian Air Force,NaN,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,98.0,2.0,"While on approach, the military transport cras...",111.53,-7.63106
5264,05/26/2009,NaN,"Near Isiro, DemocratiRepubliCongo",Service Air,NaN,Goma - Isiro,Antonov An-26,9Q-CSA,5005,4.0,4.0,NaN,The cargo plane crashed while on approach to I...,27.6083,2.77224
5265,06/01/2009,00:15,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,447,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,228.0,0.0,The Airbus went missing over the AtlantiOcean ...,-35.2009,-5.77926
5266,06/07/2009,08:30,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,NaN,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,1.0,0.0,The air ambulance crashed into hills while att...,-56.3002,52.5433


In [9]:
addresses.to_json(r'C:\Users\sueal\OneDrive\Desktop\Project2\planeData\crashes.json')

In [10]:
empty = addresses.loc[addresses["lat"] == "", :]

In [11]:
empty

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,long,lat
13,03/30/1917,NaN,Off Northern Germany,Military - German Navy,NaN,NaN,Schutte-Lanz S-L-9 (airship),NaN,NaN,23.0,23.0,0.0,Struck by lightning and crashed into the Balti...,,
16,08/21/1917,07:00,Off western Denmark,Military - German Navy,NaN,NaN,Zeppelin L-23 (airship),NaN,NaN,18.0,18.0,0.0,Shot down by British aircraft.,,
102,07/03/1926,NaN,"Rossaugpt, Czechoslovakia",Compagnie Internationale de Navigation Aérienne,NaN,Paris - Prague,Caudron C-61,F-AFBT,5307,7.0,7.0,0.0,Crashed while en route.,,
109,11/11/1926,NaN,"Cape Bojador, Morocco",Grands Express Aeriens,NaN,NaN,Breguet 14,NaN,NaN,3.0,3.0,0.0,NaN,,
237,04/14/1931,NaN,"Near Letschen, Germany",Deutsche Lufthansa,NaN,NaN,Messerschmitt M-20,D-1928,442,10.0,2.0,0.0,Fuselage failure. Four were seriously injured ...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5168,08/26/2007,16:00,"Kongolo, DemocratiRepubliCongo",Great Lakes Business Complany,NaN,Kongolo - Goma,Antonov AN-32,9Q-CAC,14-07,15.0,14.0,0.0,"The cargo plane, carrying tin ore, lost one of...",,
5193,03/15/2008,08:15,"Nbagu, Nigeria",Trade Wings Aviation Ltd.,NaN,Lagos - Bebi Air Strip,Beechcraft 1900D,5N-JAH,UE-322,3.0,3.0,0.0,The plane crashed while en route. Wreckage was...,,
5222,08/06/2008,11:40,"Sunat Tanon, Thailand",Military - Royal Thai Air Force,NaN,Betong -,Bell Huey UH-1H,NaN,NaN,10.0,10.0,0.0,The helicopter crashed while ferrying a forens...,,
5259,04/17/2009,10:30,"Mount Gergaji, Indonesia",Mimika Air,514,Ilaga - Mulia,Pilatus PC-6,PK-LTJ,959,11.0,11.0,0.0,The passenger plane crashed in poor weather in...,,


In [13]:
empty.to_json(r'C:\Users\sueal\OneDrive\Desktop\Project2\planeData\missing.json')

In [17]:
# Divide date into 3 parts
addresses[['month','day','year']] = addresses.Date.str.split('/', expand=True)
addresses

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,long,lat,month,day,year
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",-77.0811,38.8835,09,17,1908
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,-74.4229,39.3643,07,12,1912
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...,-123.366,48.4284,08,06,1913
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...,3.51562,56.511,09,09,1913
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...,13.504,52.4368,10,17,1913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,05/20/2009,06:30,"Near Madiun, Indonesia",Military - Indonesian Air Force,NaN,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,98.0,2.0,"While on approach, the military transport cras...",111.53,-7.63106,05,20,2009
5264,05/26/2009,NaN,"Near Isiro, DemocratiRepubliCongo",Service Air,NaN,Goma - Isiro,Antonov An-26,9Q-CSA,5005,4.0,4.0,NaN,The cargo plane crashed while on approach to I...,27.6083,2.77224,05,26,2009
5265,06/01/2009,00:15,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,447,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,228.0,0.0,The Airbus went missing over the AtlantiOcean ...,-35.2009,-5.77926,06,01,2009
5266,06/07/2009,08:30,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,NaN,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,1.0,0.0,The air ambulance crashed into hills while att...,-56.3002,52.5433,06,07,2009


In [19]:
# Select useful columns
cleaned_df = addresses.loc[:, ["Location", "Operator", "Aboard", "Fatalities", "Summary", "long", "lat", "year"]]
cleaned_df.tail()

,Location,Operator,Aboard,Fatalities,Summary,long,lat,year
5263,"Near Madiun, Indonesia",Military - Indonesian Air Force,112.0,98.0,"While on approach, the military transport cras...",111.53,-7.63106,2009
5264,"Near Isiro, DemocratiRepubliCongo",Service Air,4.0,4.0,The cargo plane crashed while on approach to I...,27.6083,2.77224,2009
5265,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,228.0,228.0,The Airbus went missing over the AtlantiOcean ...,-35.2009,-5.77926,2009
5266,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,1.0,1.0,The air ambulance crashed into hills while att...,-56.3002,52.5433,2009
5267,"State of Arunachal Pradesh, India",Military - Indian Air Force,13.0,13.0,The military transport went missing while en r...,94.7278,28.218,2009
